In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the CSV file
df = pd.read_csv("../data/CPS_Clean1.csv")

In [3]:
# Remove all rows for which OCC_1 equals 0
df = df[df["OCC_1"] != 0]

# Total number of rows after filtering
n_instances = len(df)

In [4]:
# Remove columns ending with '_2'
cols_to_drop = [col for col in df.columns if col.endswith('_2')]
df.drop(columns=cols_to_drop, inplace=True)

# Rename columns ending with '_1' by removing the suffix
df.rename(columns=lambda x: x[:-2] if x.endswith('_1') else x, inplace=True)

In [5]:
df.drop(columns=["ASIAN", "DURUNEMP", "WHYUNEMP", "WHYABSNT", "WNFTLOOK", 
                 "WKSUNEM1", "NWLOOKWK", "WANTJOB", "WNLWNILF", "INCRETIR", 
                 "HOURWAGE", "PAIDHOUR", "UNION", "EARNWEEK", "POPSTAT"], inplace=True, errors='ignore')

In [6]:
# Fill missing values with the mode for specified categorical columns
mode_cols = ["VETSTAT", "WKSTAT", "CLASSWLY", "FULLPART", "PENSION"]
for col in mode_cols:
    if col in df.columns:
        mode_val = df[col].mode(dropna=True)
        if not mode_val.empty:
            df[col] = df[col].fillna(mode_val.iloc[0])

# Fill missing values with the median for specified numerical columns
median_cols = ["UHRSWORKT", "UHRSWORK1", "UHRSWORKLY", "FIRMSIZE", "NUMEMPS"]
for col in median_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Remove instances where BPL or NATIVITY are missing
df.dropna(subset=["BPL", "NATIVITY"], inplace=True)

df["WKXPNS"] = df["WKXPNS"].fillna(0)

# Fill missing SCHLCOLL values by mapping the modal value for that row's AGE
age_mode = df.groupby("AGE")["SCHLCOLL"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df["SCHLCOLL"] = df.apply(lambda row: age_mode[row["AGE"]] if pd.isna(row["SCHLCOLL"]) else row["SCHLCOLL"], axis=1)
df.loc[(df["AGE"] > 50) & (df["SCHLCOLL"].isna()), "SCHLCOLL"] = 5
df.loc[(df["AGE"] == 15) & (df["SCHLCOLL"].isna()), "SCHLCOLL"] = 2

In [7]:
# Initialize the report dictionary
report = {}

# Loop through each column to compute the min, max, and median
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        min_val = df[col].min()
        max_val = df[col].max()
        median_val = df[col].median()
    else:
        min_val = max_val = median_val = "Not Applicable"
        
    # Compute missing count: NaN always; additionally, for object dtypes, consider empty strings as missing.
    missing = df[col].isna()
    if pd.api.types.is_string_dtype(df[col]):
        missing |= (df[col].str.strip() == '')
    missing_count = missing.sum()
    
    report[col] = {
        "min": min_val,
        "max": max_val,
        "median": median_val,
        "missing_percent": (missing_count / n_instances) * 100
    }

# Create a DataFrame from the report dictionary and print the report
report_df = pd.DataFrame(report).T
pd.set_option('display.max_rows', 202)
print(report_df)

                       min           max        median  missing_percent
YEAR          2.009000e+03  2.022000e+03  2.015000e+03         0.000000
SERIAL        1.000000e+00  9.934100e+04  4.551900e+04         0.000000
MONTH         3.000000e+00  3.000000e+00  3.000000e+00         0.000000
CPSID         2.008120e+13  2.022031e+13  2.014120e+13         0.000000
ASECFLAG      1.000000e+00  1.000000e+00  1.000000e+00         0.000000
ASECWTH       9.727000e+01  1.941456e+04  1.681290e+03         0.000000
PERNUM        1.000000e+00  1.600000e+01  1.000000e+00         0.000000
CPSIDP        2.008120e+13  2.022031e+13  2.014120e+13         0.000000
CPSIDV        2.008120e+14  2.022031e+14  2.014120e+14         0.000000
ASECWT        9.594000e+01  3.359769e+04  1.705505e+03         0.000000
RELATE        1.010000e+02  1.260000e+03  1.010000e+02         0.000000
AGE           1.500000e+01  8.500000e+01  4.500000e+01         0.000000
SEX           1.000000e+00  2.000000e+00  1.000000e+00         0

In [9]:
# Define the adjustment factors by YEAR
adjustment_factors = {
    2009: 1.42,
    2010: 1.397,
    2011: 1.355,
    2012: 1.327,
    2013: 1.308,
    2014: 1.287,
    2015: 1.286,
    2016: 1.27,
    2017: 1.243,
    2018: 1.213,
    2019: 1.192,
    2020: 1.177,
    2021: 1.124,
    2022: 1.041,
    2023: 1
}

# Create a Series for the adjustment factor corresponding to each row's YEAR
adjustment_series = df["YEAR"].map(adjustment_factors)

# Multiply all columns whose name contains "INC" by the adjustment factor, exempting specific columns
exempt_inc = {"INCPER", "INCPER_DELTA"}
for col in df.columns:
    if "INC" in col and col not in exempt_inc and pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col] * adjustment_series

In [10]:
# Define a list of feature exceptions that should not be transformed
exceptions = {"YEAR", "SERIAL", "MONTH", "CPSID", "ASECFLAG", 
              "ASECWTH", "PERNUM", "CPSIDP", "CPSIDV", "ASECWT", "OCC", "INCPER", "INCPER_DELTA"}

# Loop through each column in the DataFrame
for col in df.columns:
    if col in exceptions:
        continue  # Skip the exceptions
    
    # Check if the column is numeric
    if pd.api.types.is_numeric_dtype(df[col]):
        col_min = df[col].min()
        # If the minimum is greater than 0, subtract it from the column
        if col_min > 0:
            df[col] = df[col] - col_min

In [11]:
# Precompute the scaling factor for INCTOT if available
inctot_scale = None
if "INCTOT" in df.columns and pd.api.types.is_numeric_dtype(df["INCTOT"]):
    positive_vals = df["INCTOT"][df["INCTOT"] > 0]
    if not positive_vals.empty:
        pos_median = positive_vals.median()
        pos_max = positive_vals.max()
        if pos_median == 0:
            pos_median = positive_vals.mean()
            pos_max = positive_vals.max()
        if pos_median:
            if pos_median == pos_max:
                inctot_scale = 1 / pos_median
            else:
                inctot_scale = 0.5 / pos_median

# Loop through each column in the DataFrame for scaling
for col in df.columns:
    if col in exceptions:
        continue  # Skip exceptions

    # Process only numeric columns
    if pd.api.types.is_numeric_dtype(df[col]):
        # If column name contains "INC", use the INCTOT scaling factor if available
        if "INC" in col:
            if inctot_scale is not None:
                df[col] = df[col] * inctot_scale
        else:
            # Compute the scaling factor individually for non-"INC" columns
            positive_vals = df[col][df[col] > 0]
            if not positive_vals.empty:
                pos_median = positive_vals.median()
                pos_max = positive_vals.max()
                if pos_median == 0:
                    pos_median = positive_vals.mean()
                    pos_max = positive_vals.max()
                if pos_median:
                    if pos_median == pos_max:
                        scaling_factor = 1 / pos_median
                    else:
                        scaling_factor = 0.5 / pos_median
                    df[col] = df[col] * scaling_factor

In [12]:
# Fill missing values in the specified columns with -0.5
columns_to_fill = ["ELDCH", "YNGCH", "YRIMMIG"]
df[columns_to_fill] = df[columns_to_fill].fillna(-0.5)

In [13]:
# Initialize the report dictionary
report = {}

# Loop through each column to compute the min, max, and median
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        min_val = df[col].min()
        max_val = df[col].max()
        median_val = df[col].median()
    else:
        min_val = max_val = median_val = "Not Applicable"
        
    # Compute missing count: NaN always; additionally, for object dtypes, consider empty strings as missing.
    missing = df[col].isna()
    if pd.api.types.is_string_dtype(df[col]):
        missing |= (df[col].str.strip() == '')
    missing_count = missing.sum()
    
    report[col] = {
        "min": min_val,
        "max": max_val,
        "median": median_val,
        "missing_percent": (missing_count / n_instances) * 100
    }

# Create a DataFrame from the report dictionary
report_df = pd.DataFrame(report).T

# Set options to display decimals with 1 decimal place
pd.set_option('display.max_rows', 202)
pd.set_option('display.float_format', '{:.2f}'.format)

print(report_df)

                            min                max             median  \
YEAR                    2009.00            2022.00            2015.00   
SERIAL                     1.00           99341.00           45519.00   
MONTH                      3.00               3.00               3.00   
CPSID         20081200264100.00  20220306852700.00  20141204004700.00   
ASECFLAG                   1.00               1.00               1.00   
ASECWTH                   97.27           19414.56            1681.29   
PERNUM                     1.00              16.00               1.00   
CPSIDP        20081200264101.00  20220306852701.00  20141204004702.50   
CPSIDV       200812002641011.00 202203068527011.00 201412040047026.00   
ASECWT                    95.94           33597.69            1705.51   
RELATE                     0.00               5.74               0.00   
AGE                        0.00               1.17               0.50   
SEX                        0.00               1.00 

In [16]:
# Define the exception columns
# save_exceptions = {"YEAR", "SERIAL", "MONTH", "CPSID", "ASECFLAG", 
#               "ASECWTH", "PERNUM", "CPSIDP", "CPSIDV", "ASECWT", "INCCHANGE"}

save_exceptions = {"INCCHANGE", "SERIAL", "MONTH", "CPSID", "ASECFLAG", 
               "ASECWTH", "PERNUM", "CPSIDP", "CPSIDV", "ASECWT"}

# Create a new DataFrame with columns not in exceptions
df_to_save = df[[col for col in df.columns if col not in save_exceptions]]

# Save the DataFrame to a CSV file
df_to_save.to_csv("../data/occ_emb_v2.csv", index=False)